In [1]:
using Pkg

In [2]:
pkg"add DataFrames CSV Feather SQLite JLD"

   Updating registry at `/opt/julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 %

  Resolving package versions...
   Updating `/opt/julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `/opt/julia/environments/v1.4/Manifest.toml`
 [no changes]


### Download Test Data

In [3]:
download("https://nyc-tlc.s3.amazonaws.com/trip+data/green_tripdata_2019-12.csv", 
    "test_data.csv")

"test_data.csv"

In [4]:
filesize("test_data.csv")/1024^2 # in MB

39.241610527038574

## Import Test Data

In [5]:
using CSV
using DataFrames
using Dates

In [6]:
@time df = CSV.File("test_data.csv") |> DataFrame

 22.436559 seconds (13.80 M allocations: 908.744 MiB, 5.03% gc time)


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID
,Int64?,String,String,String?,Int64?
1,1,2019-12-01 00:09:45,2019-12-01 00:10:59,N,1
2,2,2019-12-01 00:26:05,2019-12-01 00:31:30,N,1
3,2,2019-12-01 00:56:36,2019-12-01 00:59:38,N,1
4,2,2019-12-01 00:26:20,2019-12-01 00:40:19,N,1
5,2,2019-12-01 00:56:36,2019-12-01 00:59:56,N,1
6,1,2019-12-01 00:14:28,2019-12-01 00:19:39,N,1
7,1,2019-12-01 00:45:54,2019-12-01 00:52:46,N,1
8,2,2019-12-01 00:25:35,2019-12-01 01:04:08,N,1
9,1,2019-12-01 00:43:12,2019-12-01 00:56:44,N,1


In [7]:
df[!, :lpep_pickup_datetime] .= DateTime.(df[!, :lpep_pickup_datetime], dateformat"yyyy-mm-dd HH:MM:SS");
df[!, :lpep_dropoff_datetime] .= DateTime.(df[!, :lpep_dropoff_datetime], dateformat"yyyy-mm-dd HH:MM:SS");

In [8]:
select!(df, Not(:ehail_fee));

## Timings

In [9]:
using BenchmarkTools

### CSV

In [10]:
outfile = "test_out.csv"

"test_out.csv"

In [11]:
@btime CSV.write(outfile, df)

  6.028 s (48926272 allocations: 1.38 GiB)


"test_out.csv"

In [12]:
@btime dfx = CSV.File(outfile) |> DataFrame;

  301.417 ms (451170 allocations: 218.88 MiB)


In [13]:
filesize(outfile)/1024^2 # in MB 

43.16948127746582

In [14]:
run(`gzip $(pwd())/$outfile`)

Process(`gzip /home/jovyan/julia_tutorial/notebooks/test_out.csv`, ProcessExited(0))

In [15]:
filesize("$outfile.gz")/1024^2 # in MB 

7.610500335693359

In [17]:
rm("$outfile.gz")

### JDF

In [54]:
using JDF

In [55]:
outfile = "test_out.jdf"

"test_out.jdf"

In [56]:
@btime savejdf(outfile, df)

  133.904 ms (1114 allocations: 106.15 MiB)


JDFFile{String}("test_out.jdf")

In [57]:
@btime dfx = loadjdf(outfile);

  81.089 ms (361171 allocations: 137.64 MiB)


In [58]:
function dirsize(outfile)
    size = 0.0
    for (root, dir, file) in walkdir(outfile)
        size += sum(filesize.(outfile, file))
    end
    size / 1024^2
end

dirsize (generic function with 1 method)

In [59]:
dirsize(outfile)

15.120265007019043

In [60]:
run(`gzip -r $(pwd())/$outfile`)

Process(`gzip -r /home/jovyan/julia_tutorial/notebooks/test_out.jdf`, ProcessExited(0))

In [61]:
dirsize(outfile)

10.351659774780273

In [62]:
rm(outfile, recursive=true)

### SQLite

In [45]:
using SQLite

In [46]:
outfile = "test_out.db"

"test_out.db"

In [47]:
db = SQLite.DB(outfile)

SQLite.DB("test_out.db")

In [48]:
@time SQLite.load!(df, db, "taxi")

 24.021676 seconds (56.59 M allocations: 2.394 GiB, 2.80% gc time)


"taxi"

In [49]:
@btime dfx = DBInterface.execute(db, "select * from taxi") |> DataFrame

  37.077 s (60010060 allocations: 2.71 GiB)


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID
,Int64?,DateTime,DateTime,String?,Int64?
1,1,2019-12-01T00:09:45,2019-12-01T00:10:59,N,1
2,2,2019-12-01T00:26:05,2019-12-01T00:31:30,N,1
3,2,2019-12-01T00:56:36,2019-12-01T00:59:38,N,1
4,2,2019-12-01T00:26:20,2019-12-01T00:40:19,N,1
5,2,2019-12-01T00:56:36,2019-12-01T00:59:56,N,1
6,1,2019-12-01T00:14:28,2019-12-01T00:19:39,N,1
7,1,2019-12-01T00:45:54,2019-12-01T00:52:46,N,1
8,2,2019-12-01T00:25:35,2019-12-01T01:04:08,N,1
9,1,2019-12-01T00:43:12,2019-12-01T00:56:44,N,1


In [50]:
filesize(outfile) / 1024^2

196.10546875

In [51]:
run(`gzip $(pwd())/$outfile`)

Process(`gzip /home/jovyan/julia_tutorial/notebooks/test_out.db`, ProcessExited(0))

In [52]:
filesize("$outfile.gz")/1024^2 # in MB 

14.859847068786621

In [53]:
rm("$outfile.gz")

### Feather

In [36]:
using Feather

In [37]:
outfile = "test_out.feather"

"test_out.feather"

In [38]:
@btime Feather.write(outfile, df)

  791.507 ms (6987 allocations: 163.36 MiB)


"test_out.feather"

In [39]:
filesize(outfile) / 1024^2

64.37792205810547

In [40]:
@btime dfx = Feather.read(outfile)

  1.794 ms (4994 allocations: 178.09 KiB)


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID
,Int64?,Arrow.T…,Arrow.T…,String?,Int64?
1,1,2019-12-01T00:09:45,2019-12-01T00:10:59,N,1
2,2,2019-12-01T00:26:05,2019-12-01T00:31:30,N,1
3,2,2019-12-01T00:56:36,2019-12-01T00:59:38,N,1
4,2,2019-12-01T00:26:20,2019-12-01T00:40:19,N,1
5,2,2019-12-01T00:56:36,2019-12-01T00:59:56,N,1
6,1,2019-12-01T00:14:28,2019-12-01T00:19:39,N,1
7,1,2019-12-01T00:45:54,2019-12-01T00:52:46,N,1
8,2,2019-12-01T00:25:35,2019-12-01T01:04:08,N,1
9,1,2019-12-01T00:43:12,2019-12-01T00:56:44,N,1


In [41]:
run(`gzip $(pwd())/$outfile`)

Process(`gzip /home/jovyan/julia_tutorial/notebooks/test_out.feather`, ProcessExited(0))

In [42]:
filesize("$outfile.gz")/1024^2 # in MB 

7.700153350830078

In [43]:
rm("$outfile.gz")

## Comparison

method | read-time (s) | write-time (s) | size (MB) | size-gzipped (MB)
-------|---------------|----------------|-----------|------------------
CSV    | 0.3           | 6              |  43       | 7.6
SQLite | 36            | 26             | 196       | 14.9
JDF    | 0.08          | 0.13           | 15.12     | 10
Feather| 0.002         | 0.79           | 64        | 7.7